In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\

In [2]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 60

In [3]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [4]:
def select_season (df, season):
    return df[df['season']==season]

In [5]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [6]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [7]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [8]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]

   
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]

    return X_train, Y_train, X_test, Y_test

In [9]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        #print(method.score(X_test, Y_test))
        #print('hi')
        lis.append(float(method.score(X_test, Y_test)))

    tmp[method_name]=lis 
    
    return tmp

In [10]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [11]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
    
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [12]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

# Loading data

In [13]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [14]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [15]:
n=4
m=4

In [16]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [17]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


## SVM_LINEAR

In [18]:
season_to_be_predicted='2015/2016'
column_to_use_as_prediction=[54,55]
column_to_predict='Goal/No_Goal'

In [19]:
mix=[allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict]

In [20]:
svm_linear = svm.SVC(kernel='linear')
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [21]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [22]:
#c=Predict_season_with_current_season(allchampionships[0].df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear')

In [23]:
a['svm_linear'].mean()

0.601955782312925

In [24]:
b['svm_linear'].mean()

0.601955782312925

## SVM_RBF


In [25]:
svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf')

In [26]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf')

In [27]:
a['svm_rbf'].mean()

0.5593537414965987

In [28]:
b['svm_rbf'].mean()

0.5599489795918368

## CART

In [29]:
dtc = DecisionTreeClassifier()
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

,dtc
0,0.400000
1,0.200000
2,0.200000
3,0.400000
4,0.600000
5,0.200000
6,0.400000
7,0.600000
8,0.600000
9,0.200000


In [30]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

,dtc
0,0.400000
1,0.200000
2,0.200000
3,0.600000
4,0.800000
5,0.200000
6,0.600000
7,0.400000
8,0.600000
9,0.200000


# Adaboost Classifier 

In [31]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

,bagb
0,0.200000
1,0.400000
2,0.400000
3,0.600000
4,0.800000
5,0.400000
6,0.400000
7,0.600000
8,0.600000
9,0.400000


In [32]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

,bagb
0,0.200000
1,0.200000
2,0.200000
3,0.600000
4,0.800000
5,0.400000
6,0.600000
7,0.400000
8,0.400000
9,0.400000


# Gradient Boosting Classifier


In [33]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=100)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

,gradb
0,0.600000
1,0.200000
2,0.200000
3,0.400000
4,0.600000
5,0.400000
6,0.400000
7,0.600000
8,0.400000
9,0.400000


In [34]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', gradb,'gradb')

NameError: name 'Italy' is not defined

# Random Forest

In [35]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
Predict_whole_season_by_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

NameError: name 'Italy' is not defined

In [ ]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

# Ridge & Lasso 

In [21]:
from sklearn.linear_model import Ridge, ridge_regression, Lasso, 

In [95]:
# using the french league
y = allchampionships[3].df['Goal/No_Goal']
X = allchampionships[3].df.iloc[:,8:-2]

In [97]:
y_results = allchampionships[3].df['FTR']

In [86]:
y_results = (allchampionships[3].df['FTR'])

In [103]:
#y_results = pd.DataFrame(allchampionships[3].df['FTR'])
for i in y_results.index:
    if y_results[i] == 'H':
        y_results[i] = 1
    if y_results[i] == 'D':
        y_results[i] = 2
    if y_results[i] == 'A':
        y_results[i] = 3

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Ridge 

In [104]:
def ridge(ALPHA, X, y):
    clf = linear_model.Ridge(alpha=ALPHA)
    clf.fit(X,y)
    tmp = pd.DataFrame(columns=X.columns)
    tmp.loc[0] = clf.coef_

    return tmp

In [112]:
ridge_df = ridge(5, X, y_results)
pd.DataFrame(ridge_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,0
H_Av_TGR,-0.144706
H_AvGS_A,-0.132849
Aw_AvGS_H,-0.085258
SUMM_av_HGSH+AGRA,-0.076788
Aw_Av_pnt_A,-0.067443
DIFF_av_TP,-0.063510
H_AvGS_H,-0.058703
H_Av_TGS,-0.033812
H_AvGR_H,-0.024203
Aw_AvGR_A,-0.018085


In [102]:
ridge_df = ridge(1, X, y)
pd.DataFrame(ridge_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,0
H_Av_TGR,-0.222370
SUMM_av_HTGR+ATGS,-0.196614
Aw_Av_T_pnt,-0.088322
SUMM_av_HTGS+ATGR,-0.077718
H_Av_T_pnt,-0.057308
H_Av_TGS,-0.045185
Aw_Av_Pnt_H,-0.039107
Aw_Av_TGR,-0.032534
Aw_AvGR_H,-0.023429
H_Av_pnt_A,-0.019672


## Lasso 

In [108]:
def lasso(ALPHA, X, y):
    #tmp = lasso_regression(X, y, alpha)
    clf = linear_model.Lasso(alpha=ALPHA)
    clf.fit(X,y)
    tmp = pd.DataFrame(columns=X.columns)
    tmp.loc[0] = clf.coef_
    #pd.DataFrame(tmp.loc[0]).sort(0)
    return tmp

In [110]:
lasso_df = lasso(0.1, X, y_results)
pd.DataFrame(lasso_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,0
H_TGS,-0.011186
Aw_TGR,-0.005825
H_T_pnt,-0.004030
H_TP5,-0.000358
H_ROUND,-0.000000
Aw_GR_H,-0.000000
Aw_GR_A,-0.000000
Aw_AvGS_H,0.000000
Aw_AvGS_A,0.000000
Aw_AvGR_H,-0.000000


In [109]:
lasso_df = lasso(0.1, X, y)
pd.DataFrame(lasso_df.loc[0]).sort(0)

C:\Users\Hrafnkell\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,0
Aw_T_pnt,-0.000400
H_ROUND,0.000000
Aw_GR_H,-0.000000
Aw_GR_A,0.000000
Aw_TGS,0.000000
Aw_TGR,0.000000
Aw_AvGS_H,-0.000000
Aw_AvGS_A,0.000000
Aw_AvGR_H,-0.000000
Aw_AvGR_A,0.000000


## Finding the percentages of outcomes in the French league 

In [30]:
home_wins = len(allchampionships[3].df[allchampionships[3].df.FTR=='H'])
draws = len(allchampionships[3].df[allchampionships[3].df.FTR=='D'])
away_wins = len(allchampionships[3].df[allchampionships[3].df.FTR=='A'])

In [36]:
home_percentage = float(home_wins) / len(allchampionships[3].df)
draw_percentage = float(draws) / len(allchampionships[3].df)
away_percentage = float(away_wins) / len(allchampionships[3].df)

In [40]:
print "Home wins: %f" %home_percentage
print "Draws: %f" %draw_percentage
print "Away wins: %f" %away_percentage

Home wins: 0.444583
Draws: 0.288333
Away wins: 0.267083


## Finding the percentages of outcomes for whole dataframe

In [44]:
home_wins = len(final[final.FTR=='H'])
draws = len(final[final.FTR=='D'])
away_wins = len(final[final.FTR=='A'])

In [45]:
home_percentage = float(home_wins) / len(final)
draw_percentage = float(draws) / len(final)
away_percentage = float(away_wins) / len(final)

In [47]:
print "Home wins: %f" %home_percentage
print "Draws: %f" %draw_percentage
print "Away wins: %f" %away_percentage

Home wins: 0.458717.2
Draws: 0.253897
Away wins: 0.287386


## All leagues 

In [53]:
number_of_leagues = len(final.country_id.unique())

In [70]:
outcomes_percentages = pd.DataFrame()
#outcomes_percentages['']

In [97]:
for i in final.country_id.unique():
    outcomes_percentages[i] = 0
outcomes_percentages.loc[1] = 0
outcomes_percentages.loc[2] = 0

In [103]:
for i in range(number_of_leagues):
    
    home = float(len(allchampionships[i].df[allchampionships[i].df.FTR=='H']))/len(allchampionships[i].df)
    draw = float(len(allchampionships[i].df[allchampionships[i].df.FTR=='D']))/len(allchampionships[i].df)
    away = float(len(allchampionships[i].df[allchampionships[i].df.FTR=='A']))/len(allchampionships[i].df)
    
    outcomes_percentages.iloc[0, i] = home
    outcomes_percentages.iloc[1, i] = draw
    outcomes_percentages.iloc[2, i] = away

In [104]:
outcomes_percentages

,Switzerland Super League,Poland Ekstraklasa,Scotland Premier League,France Ligue 1,Germany 1. Bundesliga,England Premier League,Belgium Jupiler League,Portugal Liga ZON Sagres,Netherlands Eredivisie,Italy Serie A,Spain LIGA BBVA
0,0.452830,0.454812,0.429167,0.444583,0.454885,0.460899,0.467764,0.442890,0.486371,0.466056,0.493539
1,0.243486,0.275235,0.240972,0.288333,0.243994,0.258735,0.239369,0.254662,0.230893,0.267805,0.227178
2,0.303684,0.269953,0.329861,0.267083,0.301121,0.280366,0.292867,0.302448,0.282737,0.266139,0.279283


In [106]:
outcomes_percentages.to_csv('../Data/1x2_all_leagues.csv')